In [32]:
module Systems

import Base: copy, convert
import ForwardDiff
import Flatten

macro make_frame(typename, parent, fields...)
    ex = Expr(:type, true)
    push!(ex.args, Expr(:<:))
    push!(ex.args[2].args, Expr(:curly, esc(typename), :T))
    push!(ex.args[2].args, parent)
    push!(ex.args, Expr(:block))
    for field in fields
        push!(ex.args[3].args, :($(field)::T))
    end
    return ex
end


abstract Frame
function copy{T <: Frame}(frame::T)
    T([getfield(frame, x) for x in fieldnames(frame)]...)
end

convert{T,N}(T2::Type{Array{T,N}}, f::Frame) = convert(T2, Flatten.to_vector(f))

abstract System

num_fields(T::Type) = length(fieldnames(T))
num_states(sys::System) = num_fields(state_frame(sys))
num_inputs(sys::System) = num_fields(input_frame(sys))

@make_frame DoubleIntegratorState Frame z zdot
@make_frame DoubleIntegratorInput Frame z

type DoubleIntegrator <: System
end
state_frame(sys::DoubleIntegrator) = DoubleIntegratorState
input_frame(sys::DoubleIntegrator) = DoubleIntegratorInput

dynamics{T}(sys::DoubleIntegrator, 
    t::Number, 
    x::DoubleIntegratorState{T}, 
    u::DoubleIntegratorInput{T})
    xdot = DoubleIntegratorState(x.zdot, u.z)
end

type LinearSystem <: System
    A
    B
end

num_states(sys::LinearSystem) = size(sys.A, 2)
num_inputs(sys::LinearSystem) = size(sys.B, 2)
dynamics(sys::LinearSystem, t::Number, x::Frame, u::Frame) = 
    typeof(x)((sys.A * Flatten.to_vector(x) + sys.B * Flatten.to_vector(u))...)

function dynamics(sys::System, t_x_u::Vector)
    xdot = dynamics(sys, t_x_u[1], 
    state_frame(sys)(sub(t_x_u, 1 + (1:num_states(sys)))...), 
    input_frame(sys)(sub(t_x_u, 1 + num_states(sys) + (1:num_inputs(sys)))...))
    xdot
end

type LinearSystemFactory
    sys::System
    jac::Function
end
LinearSystemFactory(sys::System) = LinearSystemFactory(sys, 
    ForwardDiff.jacobian(t_x_u::Vector -> dynamics(sys, t_x_u)))

function linearize(factory::LinearSystemFactory, t0::Number, x0::Frame, u0::Frame)
    t_x_u = [t0; Flatten.to_vector(x0); Flatten.to_vector(u0)]
    jac0 = factory.jac(t_x_u)
    A = jac0[1:end, 1+(1:num_states(factory.sys))]
    B = jac0[1:end, 1+num_states(factory.sys)+(1:num_inputs(factory.sys))]
    LinearSystem(A, B)
end

function linearize(sys::System, t0::Number, x0::Frame, u0::Frame)
    linearize(LinearSystemFactory(sys), t0, x0, u0)
end

@make_frame PendulumState Frame theta thetadot
@make_frame PendulumInput Frame tau

type Pendulum <: System
end

function dynamics(sys::Pendulum, t::Number, x::PendulumState, u::PendulumInput)
    PendulumState(x.thetadot, u.tau - sin(x.theta) - x.thetadot)
end
state_frame(sys::Pendulum) = PendulumState
input_frame(sys::Pendulum) = PendulumInput

end

Systems

In [20]:
import Systems

In [21]:
Systems.DoubleIntegratorState

Systems.DoubleIntegratorState{#108#T}

In [22]:
Flatten.to_vector(Systems.PendulumInput(0.,))

1-element Array{Float64,1}:
 0.0

In [23]:
pend = Systems.Pendulum()
p_linearized = Systems.linearize(pend, 0, Systems.PendulumState(0.,0.), Systems.PendulumInput(0.))

Systems.LinearSystem(2x2 Array{Float64,2}:
  0.0   1.0
 -1.0  -1.0,2x1 Array{Float64,2}:
 0.0
 1.0)

In [24]:
pend = Systems.Pendulum()
p_linearized = Systems.linearize(pend, 0, Systems.PendulumState(0.,0.), Systems.PendulumInput(0.))
@time for j = 1:1000; p_linearized = Systems.linearize(pend, 0, Systems.PendulumState(0.,0.), Systems.PendulumInput(0.)); end
@time factory = Systems.LinearSystemFactory(pend)
p_linearized_2 = Systems.linearize(factory, 0, Systems.PendulumState(0.,0.), Systems.PendulumInput(0.))
@time for j = 1:1000; p_linearized_2 = Systems.linearize(factory, 0, Systems.PendulumState(0.,0.), Systems.PendulumInput(0.)); end

  0.084474 seconds (241.00 k allocations: 12.405 MB, 7.55% gc time)
  0.000054 seconds (80 allocations: 4.469 KB)
  0.029585 seconds (100.00 k allocations: 5.112 MB)


In [27]:
@show Systems.dynamics(pend, 0, Systems.PendulumState(0.01, 0.01), Systems.PendulumInput(0.))
@show Systems.dynamics(p_linearized, 0, Systems.PendulumState(0.01, 0.01), Systems.PendulumInput(0.))

Systems.dynamics(pend,0,Systems.PendulumState(0.01,0.01),Systems.PendulumInput(0.0)) = Systems.PendulumState{Float64}(0.01,-0.019999833334166665)
Systems.dynamics(p_linearized,0,Systems.PendulumState(0.01,0.01),Systems.PendulumInput(0.0)) = Systems.PendulumState{Float64}(0.01,-0.02)

Systems.PendulumState{Float64}(0.01,-0.02)

In [30]:
sys = Systems.DoubleIntegrator()
x = Systems.DoubleIntegratorState(0.0, 1.0)
u = Systems.DoubleIntegratorInput(2.0)

xdot = Systems.copy(x)
@time for j = 1:1e3; xdot = Systems.dynamics(sys, 0, x, u); end
xdot

Systems.DoubleIntegratorState{Float64}(1.0,2.0)

0.000045 seconds (1000 allocations: 31.250 KB)


In [9]:
import ForwardDiff

In [10]:
sys = DoubleIntegrator()
function do_dynamics{T}(t_x_u::Vector{T})
    t = t_x_u[1]
    x = DoubleIntegratorState(t_x_u[2:3]...)
    u = DoubleIntegratorInput(t_x_u[4])
    y = toVector(dynamics(sys, t, x, u))
end
    
do_dynamics([1, 2, 3, 4])

g = ForwardDiff.jacobian(do_dynamics)

LoadError: LoadError: UndefVarError: DoubleIntegrator not defined
while loading In[10], in expression starting on line 1

In [11]:
g(rand(4))

LoadError: LoadError: UndefVarError: g not defined
while loading In[11], in expression starting on line 1

In [12]:
@time for j = 1:1000; g([1,2,3,4]); end

LoadError: LoadError: UndefVarError: g not defined
while loading In[12], in expression starting on line 1

In [34]:
@which map(sin, 1:3)

map(f, A::AbstractArray{T,N}) at abstractarray.jl:1302